In [46]:
import os

<b>Accessing the .mid files for the dictionary</b>

In [ ]:
split_dr = ["dev", "test", "train"]

for split in split_dr:
    split_path = os.path.join("musicData", split)
    for composer in os.listdir(split_path):
        if composer.endswith(".DS_Store"):
            continue
        else:
            composer_path = os.path.join(split_path, composer)
            for file in os.listdir(composer_path):
                if file.endswith(".DS_Store"):
                    continue
                else:
                    file.endswith(".mid")
                    full_path = os.path.join(composer_path, file)
                    print(full_path)

<b>Analysing the features form a single - bach344.mid fiel</b>

In [48]:
import pretty_midi
path = "musicData/dev/bach/bach344.mid"
midi = pretty_midi.PrettyMIDI(path)


<b>Basic Level Feature</b>

In [49]:
# Total duration
duration = midi.get_end_time()

# Tempo change and BPM(Beat per Min)
tempo_change , bpm = midi.get_tempo_changes()

# Time signature 
time_sig = midi.time_signature_changes
time_signature = [[ts.time, ts.numerator, ts.denominator] for ts in time_sig]

# Key signature
key_sig = midi.key_signature_changes
key_signature = [[ks.time, ks.key_number, ks.mode] for ks in key_sig]

# Beat
beat = midi.get_beats()

# Number of instrumet
num_instrument = len(midi.instruments)

basic_feature = {
    "duration": duration,
    "tempo_change": tempo_change,
    "bpm": bpm,
    "time_signature": time_signature,
    "key_signature": key_signature,
    "beat": beat,
    "num_instrument": num_instrument
}

for key, value in basic_feature.items():
    print(f"{key}: {value}")

duration: 85.73697916666666
tempo_change: [0.]
bpm: [120.]
time_signature: [[np.float64(0.0), 4, 4]]
key_signature: []
beat: [ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5 16.  16.5 17.  17.5 18.  18.5 19.  19.5 20.  20.5
 21.  21.5 22.  22.5 23.  23.5 24.  24.5 25.  25.5 26.  26.5 27.  27.5
 28.  28.5 29.  29.5 30.  30.5 31.  31.5 32.  32.5 33.  33.5 34.  34.5
 35.  35.5 36.  36.5 37.  37.5 38.  38.5 39.  39.5 40.  40.5 41.  41.5
 42.  42.5 43.  43.5 44.  44.5 45.  45.5 46.  46.5 47.  47.5 48.  48.5
 49.  49.5 50.  50.5 51.  51.5 52.  52.5 53.  53.5 54.  54.5 55.  55.5
 56.  56.5 57.  57.5 58.  58.5 59.  59.5 60.  60.5 61.  61.5 62.  62.5
 63.  63.5 64.  64.5 65.  65.5 66.  66.5 67.  67.5 68.  68.5 69.  69.5
 70.  70.5 71.  71.5 72.  72.5 73.  73.5 74.  74.5 75.  75.5 76.  76.5
 77.  77.5 78.  78.5 79.  79.5 80.  80.5 81.  81.5 82.  82.5 83.  83.5
 84.  84.5 85.  85.5]
n

<b>Instrument level feature</b>

In [54]:
# looping through the instumet attributes
for i, inst in enumerate(midi.instruments):
    if i == 3:
        break
    print(f"\nInstrument: {i+1}")
    for key, value in vars(inst).items():
        print(f"{key}: {value}")


Instrument: 1
program: 6
is_drum: False
name: Track 1
notes: [Note(start=0.345052, end=0.570312, pitch=62, velocity=127), Note(start=2.889323, end=3.114583, pitch=62, velocity=127), Note(start=4.505208, end=4.730469, pitch=62, velocity=127), Note(start=5.661458, end=5.886719, pitch=74, velocity=127), Note(start=5.661458, end=6.080729, pitch=50, velocity=127), Note(start=6.355469, end=6.580729, pitch=74, velocity=127), Note(start=7.742188, end=7.967448, pitch=50, velocity=127), Note(start=8.898438, end=9.123698, pitch=74, velocity=127), Note(start=9.822917, end=10.255208, pitch=74, velocity=127), Note(start=10.286458, end=10.511719, pitch=50, velocity=127), Note(start=10.747396, end=11.191406, pitch=74, velocity=127), Note(start=11.441406, end=11.666667, pitch=74, velocity=127), Note(start=11.903646, end=12.128906, pitch=74, velocity=127), Note(start=11.903646, end=12.128906, pitch=50, velocity=127), Note(start=13.059896, end=13.285156, pitch=62, velocity=127), Note(start=13.520833, en

<b>Note level Feature </b>

In [55]:
# Extracting noes feacture from the instrument
note_rows = []
for inst_idx, inst in enumerate(midi.instruments):
    for note in inst.notes:
        duration_note = note.end - note.start
        note_rows.append({
                'instrument_index': inst_idx,
                'pitch': note.pitch,
                'velocity': note.velocity,
                'start_time': note.start,
                'end_time': note.end,
                'duration': duration_note
            })

for i, val in enumerate(note_rows):

    if i == 20:
        break
    if val["instrument_index"] == 0:
        print(val)
    else:
        break

{'instrument_index': 0, 'pitch': 62, 'velocity': 127, 'start_time': np.float64(0.3450520833333333), 'end_time': np.float64(0.5703125), 'duration': np.float64(0.22526041666666669)}
{'instrument_index': 0, 'pitch': 62, 'velocity': 127, 'start_time': np.float64(2.8893229166666665), 'end_time': np.float64(3.114583333333333), 'duration': np.float64(0.22526041666666652)}
{'instrument_index': 0, 'pitch': 62, 'velocity': 127, 'start_time': np.float64(4.505208333333333), 'end_time': np.float64(4.73046875), 'duration': np.float64(0.22526041666666696)}
{'instrument_index': 0, 'pitch': 74, 'velocity': 127, 'start_time': np.float64(5.661458333333333), 'end_time': np.float64(5.88671875), 'duration': np.float64(0.22526041666666696)}
{'instrument_index': 0, 'pitch': 50, 'velocity': 127, 'start_time': np.float64(5.661458333333333), 'end_time': np.float64(6.080729166666666), 'duration': np.float64(0.41927083333333304)}
{'instrument_index': 0, 'pitch': 74, 'velocity': 127, 'start_time': np.float64(6.3554

<b>Statistical Features</b>

In [52]:
import numpy as np


all_duration = []
all_velocity = []
all_pitch = []
total_notes = 0


for i, inst in enumerate(midi.instruments):
    total_notes += len(inst.notes)
    for note in inst.notes:
        duration_note = note.end - note.start
        all_duration.append(duration_note)
        all_velocity.append(note.velocity)
        all_pitch.append(note.pitch)

# Average note duration and velocity
avg_note_duration = np.mean(all_duration)
avg_velocity = np.mean(all_velocity)

# Pitch range
pitch_rage = max(all_pitch) - min(all_pitch)


# Tempo mean and std
tempo_change , bpm = midi.get_tempo_changes()
tempo_mean = np.mean(tempo_change)
tempo_std = np.std(tempo_change)

# Number of instrumet
num_instrument = len(midi.instruments)


sts_features = {
    "avg_note_duration": avg_note_duration,
    "avg_velocity": avg_velocity,
    "pitch_rage": pitch_rage,
    "tempo_mean": tempo_mean,
    "tempo_std": tempo_std,
    "num_instrument": num_instrument
}

for key, value in sts_features.items():
    print(f"{key}: {value}")
        

avg_note_duration: 0.2614608293397232
avg_velocity: 127.0
pitch_rage: 46
tempo_mean: 0.0
tempo_std: 0.0
num_instrument: 12
